# Bert Intent Classification
An example Intent Classification model using BERT and HuggingFace Transformers. This notebook is based on the [HuggingFace Transformers documentation](https://huggingface.co/transformers/model_doc/bert.html#bertforsequenceclassification). We can train a model to classify the intent of a user's message. For example, if a user says "I want to buy a ticket", the model should classify the intent as "buy_ticket". This is a common task in chatbots and virtual assistants.
Steps:
1. Load data
2. Tokenize data
3. Create PyTorch Dataset
4. Train model
5. Evaluate model
6. Save model


In [1]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
import torch
import config as cfg
import warnings
from transformers import TrainerCallback
from machine_learning.model_utils import get_or_create_experiment
import mlflow
import mlflow.pytorch

warnings.filterwarnings("ignore")
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print(device)

['abbreviation' 'aircraft' 'aircraft+flight+flight_no' 'airfare'
 'airfare+flight_time' 'airline' 'airline+flight_no' 'airport' 'capacity'
 'cheapest' 'city' 'distance' 'flight' 'flight+airfare' 'flight_no'
 'flight_time' 'ground_fare' 'ground_service' 'ground_service+ground_fare'
 'meal' 'quantity' 'restriction' '<unknown>']
23
Vocabulary size: 890
Index of the word 'flight': 32
[99, 100, 140, 15, 141, 142, 12, 97]
[1, 93, 86, 3, 4, 5, 28, 64, 3, 68, 23, 260, 265]
mps


 ## Data Loading
We will use the [ATIS dataset](https://www.kaggle.com/siddhadev/atis-dataset-from-ms-cntk) from Kaggle. This dataset contains 5,047 sentences belonging to 26 intents. The sentences are queries from the ATIS (Airline Travel Information System) domain, and the intents are things like "atis_flight", "atis_airfare", "atis_ground_service", etc. The dataset is split into a training set and a test set. We will use the training set to train the model and the test set to evaluate the model. We will use the pretrained Bert tokenizer and model from HuggingFace. The tokenizer will convert the text into tokens that the model can understand. The model will be trained to classify the intent of the text. We will use the BertForSequenceClassification model, which is a pretrained Bert model with a single linear classification layer on top. This model can be used for sequence classification tasks like ours.

In [2]:
# Load the pre-trained tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load the pre-trained model for sequence classification with the number of labels
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(cfg.le.classes_))
model=model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## PyTorch Dataset
The data set uses encodings from tokenizer and labels from label encoder. The data set is then used to train the model.

In [3]:
class IntentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_length)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Load data
train = cfg.train
test = cfg.test

# Assume the second column is the label and the first column is the text
train_texts = train[0].tolist()
test_texts = test[0].tolist()

# Convert labels to integer (if they are not already)
# This might involve using a LabelEncoder as you have categorical labels
from sklearn.preprocessing import LabelEncoder
label_encoder = cfg.le
num_labels = len(cfg.le.classes_)

# Tokenize the text and create datasets
max_length = 256  # Max length of the text sequence, you might need to adjust this based on your dataset
train_dataset = IntentDataset(train_texts, cfg.train_labels, tokenizer, max_length)
test_dataset = IntentDataset(test_texts, cfg.test_labels, tokenizer, max_length)

## Training
Hyperparameters are defined here. The model is trained and evaluated.

In [4]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',  
    logging_strategy="steps",  # or "epoch"
    logging_steps=50  # Log every 10 steps# directory for storing logs,
)

class MLflowLoggingCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        # Log metrics with MLflow here
        if metrics:
            for key, value in metrics.items():
                mlflow.log_metric(key, value, step=state.global_step)

try:
    # Create an experiment and log parameters
    get_or_create_experiment("BertIntentClassification Pretrained")
    mlflow(pytorch=True)
    mlflow.start_run()
    mlflow.log_param("epochs", training_args.num_train_epochs)
    mlflow.log_param("batch_size", training_args.per_device_train_batch_size)
    mlflow.log_param("learning_rate", training_args.learning_rate)
    mlflow.log_param("weight_decay", training_args.weight_decay)
    mlflow.log_param("warmup_steps", training_args.warmup_steps)
    mlflow.log_param("max_length", max_length)
    mlflow.log_param("num_labels", num_labels)
    mlflow.log_param("model", "bert-base-uncased")

except:
    pass
#mlflow.log_params(your_params_dict)  # Log any initial parameters
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    callbacks=[MLflowLoggingCallback()]
)
trainer.train()
mlflow.end_run()

Step,Training Loss
50,3.230800
100,2.187300
150,0.889900
200,0.637000
250,0.382900
300,0.300500
350,0.222600
400,0.206100
450,0.227100
500,0.125200


## Save Model

In [5]:
trainer.save_model('results/final_bert_evaluated')


 ## Evaluate Model

In [7]:
trainer.evaluate()

{'eval_loss': 0.17989060282707214,
 'eval_runtime': 0.9703,
 'eval_samples_per_second': 875.999,
 'eval_steps_per_second': 14.428,
 'epoch': 3.0}